# Noter om analyse af orienteringsløbefil

## Del 1 : Tempo i tre trin

En slags vejledning i hvordan man kommer i gang med at gøre noget ved dataerne fra løbefilerne.

Først nogle funktioner jeg har gaflet. 
Begge dele er fra nogle obscure websites. Desværre er gpxpy stort set ikke dokumenteret, Der er eller mage gode til i librariet.

Funktionen `read_gpx_file()` er fra <https://www.programcreek.com/python/example/122591/gpxpy.parse> Example 2
Exception klassen `TrackException(Exception)` fandt jeg langt nede i et git repo, <https://github.com/acsicuib/YAFS/blob/master/src/trackanimation/utils.py>

In [2]:
import gpxpy
import os

# ---------------->8
#skraldet fra https://www.programcreek.com/python/example/122591/gpxpy.parse Example 2

def read_gpx_file(filename):
    try:
        with open(filename, "r") as f:
            points_list = []
            prev_point = None
            head, tail = os.path.split(filename)
            code_route = tail.replace(".gpx", "")
            try:
                gpx = gpxpy.parse(f)
                for point in gpx.walk(only_points=True):
                    speed = point.speed_between(prev_point)
                    if speed is None:
                        speed = 0

                    time_difference = point.time_difference(prev_point)
                    if time_difference is None:
                        time_difference = 0

                    distance = point.distance_3d(prev_point)
                    if not distance:
                        distance = point.distance_2d(prev_point)
                    if distance is None:
                        distance = 0

                    points_list.append([code_route, point.latitude, point.longitude, point.elevation,
                                                point.time, speed, time_difference, distance])

                    prev_point = point
            except Exception as e:
                raise TrackException('GPX file "' + filename + '" malformed', e)
    except FileNotFoundError as e:
        raise TrackException('GPX file "' + filename + '" not found', e) 

    return points_list
# 8<-------

# 8<----  https://github.com/acsicuib/YAFS/blob/master/src/trackanimation/utils.py
class TrackException(Exception):
    """
    Generic exception for TrackAnimation
    """

    def __init__(self, msg, original_exception):
        super(TrackException, self).__init__(msg + (": %s" % original_exception))
        self.original_exception = original_exception
# ----->8


## Indlæsning og videre beregning af gpxpy fil

Alle punkterne som returneres af `read_gpx_file()` har fåe tilføjet `speed, time_difference, distance` på index `[5]`, `[6]` og `[7]` som beregnes ud fra tidsforskellen, og distancen mellem hvert punkt og det forrige punkt.  
Distancen beregnes med `gpxpy`'s  `point.distance_3d()`, som bruger en elepsoid algoritme. Måske tager den også højdefoskel mellem punkterne med?


In [3]:
filepath = r"."
filename = r"\data\activity_7403521010.gpx"
points = read_gpx_file(filepath + filename)

print(len(points))
# print(points[0])
# print(points[1])

# print(points[100])

print("points 600-609:\n[speed, time_difference, distance]")
print( *[p[5:8] for p in points[600:610]], sep='\n')

1131
points 600-609:
[speed, time_difference, distance]
[0.759343045464342, 5.0, 3.7967152273217097]
[0.6506223979435596, 3.0, 1.9518671938306786]
[0.8284450436492407, 5.0, 4.142225218246203]
[0.8889795026959079, 4.0, 3.5559180107836315]
[0.5862917088767963, 2.0, 1.1725834177535925]
[0.4494878013319975, 1.0, 0.4494878013319975]
[0.9964274270515685, 3.0, 2.9892822811547055]
[1.0610857487983982, 1.0, 1.0610857487983982]
[0.9840859616067157, 2.0, 1.9681719232134314]
[0.8924549423791625, 4.0, 3.56981976951665]


## Sum af del-stykker i de tre hastigheder

og så den gennerell sum af stykker i de tre hastigheder

### omregning ml km/m og m/s

<https://www.aqua-calc.com/convert/speed/minute-per-kilometer-to-meter-per-second>


Kan _tempo_ i min/km, omregnes til _hastighed_ i m/s, hvor $ p $ er pace, dvs. tempoet i $ \frac{min}{km} $:

$$
    p \cdot \tfrac{min}{km}  

    = p \frac{60 \cdot s}{1000 \cdot m} 
    = \frac{ \frac{1000 \cdot m}{60 \cdot s} }{p} 
    = \frac{ \frac{1000}{60} }{p} \cdot \tfrac{m}{s}
  
    = \frac{16.667}{p} \cdot \tfrac{m}{s}
$$

Altså bliver de to grænseværdier:

- løb: $ 13 \tfrac{min}{km} = 1,282051 \cdot \tfrac{m}{s} $
- gang: $ 13 \tfrac{min}{km} = 0,33333 \cdot \tfrac{m}{s} $

Vi kan også implementere funktionen `pace2velocity()`:

In [4]:
def pace2velocity(pace):
    """ Converts pace with unit minutes/km, to velocity, with unit m/s (meters pr. second)"""
    return (1000/60)/pace

print(f"13 min/km = {pace2velocity(13) :.6f} m/s")
print(f"50 min/km = {pace2velocity(50) :.6f} m/s")

13 min/km = 1.282051 m/s
50 min/km = 0.333333 m/s


Du skal gennmløbe listen point, f.eks. med et `for`-loop. I loopet, kan du lægge til variblerne herunder for at akkumulere hvor meget om hvert stykke vej mellem et punkt og det forrige, er foregået i løb eller gang, eller har være i nær-stilstand. Aka  _run_, _walk_ og _still_.  For at afgøre hvilken af af bevægelsestyperene der er tale om kan du lave en if-elif-else blok.

Husk at i hvert punkt i listen `points` er hastigheden (i m/s) i index 5 (`point[5]`), tiden i index 6 og distancen i index 7.

In [5]:
dist_walk  = 0
dist_run   = 0
dist_still = 0
dist_all   = 0
time_walk  = 0
time_run   = 0
time_still = 0
time_all   = 0

# her skal være en konstruktion der gennemløber listen `points`...

Så er det bare at udskrive pænt:

In [ ]:
print(f"                    meters, secunds,   % (of time)")
print(f"Acumulated walk,  {dist_walk  :>8.2f},   {time_walk  :5.0f}, {time_walk/time_all  :6.1%} ")
print(f"Acumulated run,   {dist_run   :>8.2f},   {time_run   :5.0f}, {time_run/time_all   :6.1%} ")
print(f"Acumulated still, {dist_still :>8.2f},   {time_still :5.0f}, {time_still/time_all :6.1%} ")
print(f"Acumulated all,   {dist_all   :>8.2f},   {time_all   :5.0f}, {time_all/time_all   :6.1%} ")

print(f"Average speed is {dist_all/time_all :2.2f}m/s {dist_all/time_all * 3.6 :2.2f}km/t, {dist_all/time_all /0.06 :2.2f}min/km ")
